In [ ]:
from tensorflow.keras import layers
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
# Select the time-to-event time
def selected_time(select_time, time, event):
    if (time > select_time): return 0
    else:
        if (event == 1.0): return 1
        else: return None

In [ ]:
survival_metric_geo='Pat_Died'
survival_time_geo='Pat_Overall_Survival_Months'
time_geo=12

In [ ]:
geo_clinical=pd.read_hdf('D:/MyData/Pathway_Attention/data/LUSC/tcga_lusc.h5',key='clinical')
test_clinical=pd.read_hdf('data/LUSC/gse74777.h5',key='clinical')

In [ ]:
geo_clinical.drop_duplicates()

In [ ]:
test_clinical.drop_duplicates()

In [ ]:
geo_categorical = geo_clinical[[survival_metric_geo, survival_time_geo]].apply(lambda row: selected_time(time_geo, row[survival_time_geo], row[survival_metric_geo]), axis=1)
geo_categorical.dropna(inplace=True)
geo_categorical.value_counts(normalize=True)

In [ ]:
test_cate= test_clinical[[survival_metric_geo, survival_time_geo]].apply(lambda row: selected_time(time_geo, row[survival_time_geo], row[survival_metric_geo]), axis=1)
test_cate.dropna(inplace=True)
test_cate.value_counts(normalize=True)

In [ ]:
df=pd.DataFrame(geo_categorical)
df['Sam_Name']=geo_categorical.index
df

In [ ]:
dft=pd.DataFrame(test_cate)
dft['Sam_Name']=test_cate.index
dft

In [ ]:
df=df.drop_duplicates(subset=['Sam_Name'],keep='first')

In [ ]:
df

In [ ]:
dft=dft.drop_duplicates(subset=['Sam_Name'],keep='first')

In [ ]:
dft

In [ ]:
clinical=geo_clinical.loc[[s for s in df.index]]

In [ ]:
clinical=clinical.drop_duplicates()
clinical

In [ ]:
test_clinical=test_clinical.loc[[s for s in dft.index]]

In [ ]:

test_clinical=test_clinical.drop_duplicates()
test_clinical

In [ ]:
%%time
#读取图像数据
dir_name = 'D:/MyData/Pathway_Attention/data/IMG/'

X = np.array([np.load(dir_name+s+".npy") for s in df[0].index]) 
X_test=np.array([np.load(dir_name+s+".npy") for s in dft[0].index]) 
'''
X=[]
for s in df[0].index:
    array0=np.load(dir_name+s+".npy")
    array1=np.stack([array0, array0, array0], axis=-1)
    X.append(array1.tolist())
X=np.array(X)

X_test=[]
for s in dft[0].index:
    array0=np.load(dir_name+s+".npy")
    array1=np.stack([array0, array0, array0], axis=-1)
    X_test.append(array1.tolist())
X_test=np.array(X_test)

print(X.shape)
print(X_test.shape)
'''

In [ ]:
print(X.shape)
print(X_test.shape)

In [ ]:
%%time
from sklearn.preprocessing import LabelEncoder
y = LabelEncoder().fit_transform(df[0])
y=y.reshape(y.shape[0],1)
y_test=LabelEncoder().fit_transform(dft[0])
y_test=y_test.reshape(y_test.shape[0],1)

In [ ]:
%%time
#划分训练集测试集
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.2,random_state=130)
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)


In [ ]:
%%time
X_random_train=X_train.reshape(X_train.shape[0],49*49)
X_i_train=[[i] for i in range(len(y_train))]
#X_smote.shape
from imblearn.over_sampling import SMOTE as sm


X_ri_train, y_r_train = sm(random_state=321).fit_resample(X_i_train,y_train)
X_r_train=X_random_train[[i[0] for i in X_ri_train]]


X_r_train=X_r_train.reshape(X_r_train.shape[0],49,49,1)

y_r_train=y_r_train.reshape(y_r_train.shape[0],1)
print(X_r_train.shape)
print(y_r_train.shape)

In [ ]:
def activation_block(x):
    x = layers.Activation("gelu")(x)
    return layers.BatchNormalization()(x)


def conv_stem(x, filters: int, patch_size=7):
    x = layers.Conv2D(filters, kernel_size=patch_size, strides=patch_size)(x)
    return activation_block(x)


def conv_mixer_block(x, filters: int, kernel_size: int):
    # Depthwise convolution.
    x0 = x
    x = layers.DepthwiseConv2D(kernel_size=kernel_size, padding="same")(x)
    x = layers.Add()([activation_block(x), x0])  # Residual.

    # Pointwise convolution.
    x = layers.Conv2D(filters, kernel_size=1)(x)
    x = layers.Activation("gelu")(x)
    x=layers.BatchNormalization()(x)
    
    return x

'''
def get_conv_mixer_343_7(
    image_size=49, filters=343, depth=7, kernel_size=3, patch_size=7, num_classes=2
):
    """ConvMixer-256/8: https://openreview.net/pdf?id=TVHS5Y4dNvM.
    The hyperparameter values are taken from the paper.
    """
    inputs = keras.Input((image_size, image_size, 3))
    #x = layers.Rescaling(scale=1.0 / 255)(inputs)
   
    # Extract patch embeddings.
    x = conv_stem(inputs, filters, patch_size)

    # ConvMixer blocks.
    for _ in range(depth):
        x = conv_mixer_block(x, filters, kernel_size)

    # Classification block.
    x = layers.GlobalAvgPool2D()(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    
    

    return keras.Model(inputs, outputs)
'''

In [ ]:
import keras_tuner as kt
def buildConvMixer(hp):
    inputs = keras.Input((49, 49, 1))
    #the conv stem, looking for extracting yhe patch embeddings
    x = layers.Conv2D(hp.Choice('filters1',[16,32,64,80,96,128,256]), kernel_size=7, strides=7)(inputs)
    x = layers.Activation('gelu')(x)
    x=layers.BatchNormalization()(x)
    
    x = conv_mixer_block(x, hp.Choice('filters1',[16,32,64,80,96,128,256]), kernel_size=hp.Choice('ks',[3,5,7]))
    #x = conv_mixer_block(x, hp.Choice('filters1',[16,32,64,80,96,128,256]), kernel_size=hp.Choice('ks',[3,5,7]))
    #x = conv_mixer_block(x, hp.Choice('filters1',[16,32,64,128,256]), kernel_size=hp.Choice('ks',[3,5,7]))
    #x = conv_mixer_block(x, hp.Choice('filters1',[32,48,64,80,96,128,256]), kernel_size=hp.Choice('ks',[3,5,7,9]))
    #x = conv_mixer_block(x, hp.Choice('filters1',[64,80,96,112,128,160,256]), kernel_size=hp.Choice('ks',[3,5,7]))

    # Classification block.
    x = layers.GlobalAvgPool2D()(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model=keras.Model(inputs,outputs)
     
    optimizer = tfa.optimizers.AdamW(learning_rate=0.001, weight_decay=0.0001)

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.BinaryCrossentropy(
            from_logits=False, label_smoothing=0.1
        ),
        metrics=[
#            keras.metrics.BinaryAccuracy(name="accuracy"),
#            keras.metrics.BinaryAccuracy(
#                name="binary_accuracy", dtype=None, threshold=0.5
#            ),
            keras.metrics.AUC(name='auc',from_logits=False),
#            keras.metrics.Precision(name='precision'),
#            keras.metrics.Recall(name='recall'),  
#            tfa.metrics.F1Score(num_classes,threshold=0.5,name='f1_score')
        ],
    )
    
    return model

In [ ]:
tuner1 = kt.BayesianOptimization(
        buildConvMixer,
        objective=kt.Objective("val_auc", direction="max"),
        max_trials=50,
        overwrite=True)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_auc',mode='max', patience=6)
tuner1.search([X_r_train],
              y_r_train,
              epochs=20,
              #batch_size=hp.Choice("batch_size", [16, 32]),
              validation_data=([X_val], y_val),
              callbacks=[stop_early])
best_model = tuner1.get_best_models()[0]

In [ ]:
best_hyperparameters = tuner1.get_best_hyperparameters(1)[0]

In [ ]:
best_model.summary()

In [ ]:
tuner1.results_summary(num_trials=1)

In [ ]:
import sklearn.metrics
y_pred=best_model.predict([X_test])
auc=sklearn.metrics.roc_auc_score(y_test,y_pred)
auc

In [ ]:

# Code reference: https://bit.ly/3awIRbP.
import matplotlib.cm as c

def visualization_plot(weights, idx=1):
    # First, apply min-max normalization to the
    # given weights to avoid isotrophic scaling.
    p_min, p_max = weights.min(), weights.max()
    weights = (weights - p_min) / (p_max - p_min)

    # Visualize all the filters.
    num_filters = 16
    plt.figure(figsize=(14, 14))

    for i in range(num_filters):
        current_weight = weights[:, :, :, i]
        if current_weight.shape[-1] == 1:
            current_weight = current_weight.squeeze()
        ax = plt.subplot(4, 4, idx)
        ax.set_xticks([])
        ax.set_yticks([])
        plt.imshow(current_weight,c.RdGy)
        
        idx += 1
    plt.savefig('D:/Mydata/实验结果和论文图片_ConvMixer/figs/feature_embeddings.svg', dpi=600, format='svg')



# We first visualize the learned patch embeddings.
patch_embeddings = best_model.layers[1].get_weights()[0]
visualization_plot(patch_embeddings)


## Cross Validation

In [ ]:
%%time
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE as os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve
import sklearn.metrics
accuracy_list=[]
auc_list=[]
precision_list=[]
recall_list=[]
f1_score_list=[]
thresh_list=[]


accuracy_list_t=[]
auc_list_t=[]
precision_list_t=[]
recall_list_t=[]
f1_score_list_t=[]
thresh_list_t=[]

init_random_state=100
for i in range(0,50):
    X_train,X_val,geo_categorical_train,geo_categorical_test,y_train,y_val=train_test_split(X,df[0],y,test_size=0.2,random_state=init_random_state+i)
    print('=============================================================================================================')
    print('* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *')
    print(' * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * ')
    print('* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *')
    print(' * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * ')
    print('* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *')
    print('=============================================================================================================')
    print('current random state is: {}'.format(init_random_state+i))
    print('This is the {}th fold'.format(i+1))
   
    X_random_train=X_train.reshape(X_train.shape[0],49*49)
    
    X_i_train=[[i] for i in range(len(y_train))]
    X_ri_train, y_r_train = os(random_state=321+i).fit_resample(X_i_train,y_train)
    X_r_train=X_random_train[[i[0] for i in X_ri_train]]
    

    X_r_train=X_r_train.reshape(X_r_train.shape[0],49,49,1)
    
    y_r_train=y_r_train.reshape(y_r_train.shape[0],1)

    model=buildConvMixer(best_hyperparameters)
    
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=8e-4,
    decay_steps=16,
    decay_rate=0.99)

    
    optimizer = tfa.optimizers.AdamW(learning_rate=0.001, weight_decay=0.0001)
    #optimizer=keras.optimizers.Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer,
        loss=keras.losses.BinaryCrossentropy(
            from_logits=False, label_smoothing=0.1
        ),
        metrics=[
#            keras.metrics.BinaryAccuracy(name="accuracy"),
#            keras.metrics.BinaryAccuracy(
#                name="binary_accuracy", dtype=None, threshold=0.5
#            ),
            keras.metrics.AUC(name='auc',from_logits=False,num_thresholds=500),
#            keras.metrics.Precision(name='precision'),
#            keras.metrics.Recall(name='recall'),  
#            tfa.metrics.F1Score(num_classes,threshold=0.5,name='f1_score')
        ],
    )

    checkpoint_filepath = "./tmp/checkpoint"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_auc",
        mode='max',
        save_best_only=True,
        save_weights_only=True,
    )
    
    early_stop = keras.callbacks.EarlyStopping(monitor='val_auc',mode='max', patience=30)

    history = model.fit(
        x=([X_r_train]),
        y=y_r_train,
        batch_size=32,
        epochs=150,
        validation_data=([X_val], y_val),
        callbacks=[checkpoint_callback,early_stop],
    )

    model.load_weights(checkpoint_filepath)
#    _, accuracy, binary_accuracy,auc_roc = model.evaluate(X_test, y_test)
    #_, accuracy,auc,precision,recall = model.evaluate([X_f_test,X_p_test,age_test,stage_test], y_test)
    y_pred=model.predict([X_val])
    auc=sklearn.metrics.roc_auc_score(y_val,y_pred)
    geo_categorical_pred=pd.DataFrame(geo_categorical_test)
    geo_categorical_pred['prediction']=y_pred
    
    fpr, tpr, thresholds = roc_curve(y_val, y_pred)
    J = tpr - fpr
    index_max = np.argmax(J)
    best_thresh = thresholds[index_max]
    
    tp=geo_categorical_pred[(geo_categorical_pred[0]==1.0) & (geo_categorical_pred['prediction']>=best_thresh)].shape[0]
    fp=geo_categorical_pred[(geo_categorical_pred[0]==1.0) & (geo_categorical_pred['prediction']<best_thresh)].shape[0]
    tn=geo_categorical_pred[(geo_categorical_pred[0]==0.0) & (geo_categorical_pred['prediction']<=best_thresh)].shape[0]
    fn=geo_categorical_pred[(geo_categorical_pred[0]==0.0) & (geo_categorical_pred['prediction']>best_thresh)].shape[0]
    
    accuracy=(tp+tn)/(tp+fp+tn+fn)
    precision=tp/(tp+fp)
    recall=tp/(tp+fn)
    f1_score=2*(precision*recall)/(precision+recall) 
    
#External Validation / Generalization Validation   
    print('External Validation, waiting...')
    y_pred_t=model.predict([X_test])
    auc_t=sklearn.metrics.roc_auc_score(y_test,y_pred_t)
    df_pred=pd.DataFrame(dft[0])
    df_pred['prediction']=y_pred_t
    
    fpr_t, tpr_t, thresholds_t = roc_curve(y_test, y_pred_t)
    J_t = tpr_t - fpr_t
    index_max_t = np.argmax(J_t)
    best_thresh_t = thresholds_t[index_max_t]
    
    tp_t=df_pred[(df_pred[0]==1.0) & (df_pred['prediction']>=best_thresh_t)].shape[0]
    fp_t=df_pred[(df_pred[0]==1.0) & (df_pred['prediction']<best_thresh_t)].shape[0]
    tn_t=df_pred[(df_pred[0]==0.0) & (df_pred['prediction']<=best_thresh_t)].shape[0]
    fn_t=df_pred[(df_pred[0]==0.0) & (df_pred['prediction']>best_thresh_t)].shape[0]
    
    accuracy_t=(tp_t+tn_t)/(tp_t+fp_t+tn_t+fn_t)
    precision_t=tp_t/(tp_t+fp_t)
    recall_t=tp_t/(tp_t+fn_t)
    f1_score_t=2*(precision_t*recall_t)/(precision_t+recall_t) 
    '''
    y_pred_geo=model.predict([X_f_geo])
    auc_geo=sklearn.metrics.roc_auc_score(y_geo,y_pred_geo)
    geo_categorical_pred=pd.DataFrame(geo_categorical)
    geo_categorical_pred['prediction']=y_pred_geo
    
    fpr_geo, tpr_geo, thresholds_geo = roc_curve(y_geo, y_pred_geo)
    J_geo = tpr_geo - fpr_geo
    index_max_geo = np.argmax(J_geo)
    best_thresh_geo = thresholds_geo[index_max_geo]
    
    tp_geo=geo_categorical_pred[(geo_categorical_pred[0]==1.0) & (geo_categorical_pred['prediction']>=best_thresh_geo)].shape[0]
    fp_geo=geo_categorical_pred[(geo_categorical_pred[0]==1.0) & (geo_categorical_pred['prediction']<best_thresh_geo)].shape[0]
    tn_geo=geo_categorical_pred[(geo_categorical_pred[0]==0.0) & (geo_categorical_pred['prediction']<=best_thresh_geo)].shape[0]
    fn_geo=geo_categorical_pred[(geo_categorical_pred[0]==0.0) & (geo_categorical_pred['prediction']>best_thresh_geo)].shape[0]
    
    accuracy_geo=(tp_geo+tn_geo)/(tp_geo+fp_geo+tn_geo+fn_geo)
    precision_geo=tp_geo/(tp_geo+fp_geo)
    recall_geo=tp_geo/(tp_geo+fn_geo)
    f1_score_geo=2*(precision_geo*recall_geo)/(precision_geo+recall_geo) 
    '''
   
    #print(f"Test accuracy: {round(accuracy * 100, 2)}%")
#    print(f"Test binary accuracy: {round(binary_accuracy * 100, 2)}%")
    #print(f"Test AUC: {round(auc, 4)}")
    #print(f"Test precision: {round(precision * 100, 2)}%")
    #print(f"Test recall: {round(recall * 100, 2)}%")
    #print(f"Test f1_score: {round(f1_score*100,2)}%")
    accuracy_list.append(accuracy)
    auc_list.append(auc)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_score_list.append(f1_score)
    thresh_list.append(best_thresh)
    
    accuracy_list_t.append(accuracy_t)
    auc_list_t.append(auc_t)
    precision_list_t.append(precision_t)
    recall_list_t.append(recall_t)
    f1_score_list_t.append(f1_score_t)
    thresh_list_t.append(best_thresh_t)
    '''
    accuracy_list_geo.append(accuracy_geo)
    auc_list_geo.append(auc_geo)
    precision_list_geo.append(precision_geo)
    recall_list_geo.append(recall_geo)
    f1_score_list_geo.append(f1_score_geo)
    thresh_list_geo.append(best_thresh_geo)
    '''
print('mean accuracy={}±{}'.format(np.mean(accuracy_list),np.std(accuracy_list)))
print('mean auc={}±{}'.format(np.mean(auc_list),np.std(auc_list)))
print('mean precision={}±{}'.format(np.mean(precision_list),np.std(precision_list)))
print('mean recall={}±{}'.format(np.mean(recall_list),np.std(recall_list)))
print('mean f1_score={}±{}'.format(np.mean(f1_score_list),np.std(f1_score_list)))
print('mean threshold={}±{}'.format(np.mean(thresh_list),np.std(thresh_list)))

print('=========================================================')

print('mean accuracy independent={}±{}'.format(np.mean(accuracy_list_t),np.std(accuracy_list_t)))
print('mean auc independent={}±{}'.format(np.mean(auc_list_t),np.std(auc_list_t)))
print('mean precision independent={}±{}'.format(np.mean(precision_list_t),np.std(precision_list_t)))
print('mean recall independent={}±{}'.format(np.mean(recall_list_t),np.std(recall_list_t)))
print('mean f1_score independent={}±{}'.format(np.mean(f1_score_list_t),np.std(f1_score_list_t)))
print('mean threshold independent={}±{}'.format(np.mean(thresh_list_t),np.std(thresh_list_t)))
'''
print('GEO mean accuracy={}±{}'.format(np.mean(accuracy_list_geo),np.std(accuracy_list_geo)))
print('GEO mean auc={}±{}'.format(np.mean(auc_list_geo),np.std(auc_list_geo)))
print('GEO mean precision={}±{}'.format(np.mean(precision_list_geo),np.std(precision_list_geo)))
print('GEO mean recall={}±{}'.format(np.mean(recall_list_geo),np.std(recall_list_geo)))
print('GEO mean f1_score={}±{}'.format(np.mean(f1_score_list_geo),np.std(f1_score_list_geo)))
print('GEO mean threshold={}±{}'.format(np.mean(thresh_list_geo),np.std(thresh_list_geo)))
'''

In [ ]:
f1_score_list

In [ ]:
recall_list_t

In [ ]:
list0=[0.529320987654321,
 0.6337448559670782,
 0.6219135802469136,
 0.5895061728395061,
 0.6136831275720165,
 0.6064814814814814,
 0.5905349794238683,
 0.49691358024691357,
 0.5730452674897119,
 0.6105967078189299,
 0.5601851851851851,
 0.5452674897119341,
 0.5648148148148148,
 0.6165123456790124,
 0.6075102880658436,
 0.5524691358024691,
 0.6219135802469136,
 0.6224279835390946,
 0.5889917695473251,
 0.6604938271604938,
 0.5447530864197531,
 0.5159465020576132,
 0.5385802469135801,
 0.5128600823045267,
 0.5910493827160495,
 0.5879629629629629,
 0.573559670781893,
 0.5619855967078189,
 0.5545267489711935,
 0.6136831275720165,
 0.6033950617283951,
 0.558641975308642,
 0.6039094650205761,
 0.5524691358024691,
 0.5339506172839507,
 0.5406378600823045,
 0.5679012345679012,
 0.5267489711934157,
 0.5679012345679012,
 0.5874485596707819,
 0.4984567901234568,
 0.5843621399176955,
 0.5658436213991769,
 0.5740740740740741,
 0.5740740740740741,
 0.5951646090534979,
 0.6039094650205762,
 0.587962962962963,
 0.48919753086419754,
 0.5210905349794238]

In [ ]:
np.mean(list0)


## Grad-Cam

In [ ]:
from IPython.display import Image, display

In [ ]:
img_size = (49, 49)

In [ ]:
last_conv_layer_name = "conv2d_2"

In [ ]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 49x49
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (49, 49, 1)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 49, 49, 1)
    array = np.expand_dims(array, axis=0)
    return array


In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
img_array=X.reshape(X.shape[0],49,49,1)[0]
img_array.shape

In [ ]:
import matplotlib.cm as c
plt.figure(figsize=(10, 10))
plt.xticks([])  #去掉横坐标值
plt.yticks([])  #去掉纵坐标值
plt.imshow(img_array[0:7,0:7],c.Spectral)
plt.savefig('D:/Mydata/实验结果和论文图片_ConvMixer/figs/pathway_block.svg', dpi=600, format='svg')

In [ ]:
img_array_t=X_test.reshape(X_test.shape[0],49,49,1)[0]
img_array_t.shape

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(img_array_t[8:14,0:7],c.PiYG)
plt.colorbar()

In [ ]:
img_array=img_array.reshape(1,49,49,1)
model=best_model
last_conv_layer_name='conv2d_2'
pred_index=0
model.layers[-1].activation=None

In [ ]:
# Generate class activation heatmap
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name,pred_index)

# Display heatmap
plt.imshow(heatmap,c.PiYG)
plt.colorbar()

In [ ]:
best_model.predict(img_array)

In [ ]:
df

In [ ]:
df.columns=['label','Sam_Name']

In [ ]:
df_positive=df.loc[df.label==1.0]
df_positive

In [ ]:
df_negative=df.loc[df.label==0.0]
df_negative

In [ ]:
X_positive = np.array([np.load(dir_name+s+".npy") for s in df_positive.index]) 
X_negative = np.array([np.load(dir_name+s+".npy") for s in df_negative.index])
print(X_positive.shape)
print(X_negative.shape)

In [ ]:
%%time
heatmap_negative=[]
for i in range(X_negative.shape[0]):
    img_array=X_negative[i]
    img_array = np.expand_dims(img_array, axis=0)
    model=best_model
    model.layers[-1].activation=None
    heatmap= make_gradcam_heatmap(img_array, model, last_conv_layer_name)
    heatmap_negative.append(heatmap)
    
heatmap_negative=np.array(heatmap_negative)
print(heatmap_negative.shape)

In [ ]:
hm_mean_n=np.mean(heatmap_negative,axis=0)

In [ ]:
hm_mean_n.shape

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(hm_mean_n,c.Reds)
plt.xticks([])  #去掉横坐标值
plt.yticks([])  #去掉纵坐标值

In [ ]:
%%time
heatmap_positive=[]
for i in range(X_positive.shape[0]):
    img_array=X_positive[i]
    img_array = np.expand_dims(img_array, axis=0)
    model=best_model
    model.layers[-1].activation=None
    heatmap= make_gradcam_heatmap(img_array, model, last_conv_layer_name)
    heatmap_positive.append(heatmap)
    
heatmap_positive=np.array(heatmap_positive)
print(heatmap_positive.shape)

In [ ]:
hm_mean_p=np.mean(heatmap_positive,axis=0)
plt.imshow(hm_mean_p,c.PiYG)
plt.colorbar()

## Key Pathways Finding

In [ ]:
from scipy import stats

In [ ]:
df_hm_p=pd.DataFrame(heatmap_positive.reshape(heatmap_positive.shape[0],49))


df_hm_p.columns=['Androgen_receptor_signaling_pathway','Cytokine-Jak-STAT_signaling_pathway','DCC-apoptotic_pathway','EGF-EGFR-PI3K_signaling_pathway','EGF-EGFR-PLCG-CAMK_signaling_pathway','EGF-EGFR-RAS-ERK_signaling_pathway','EGF-EGFR-RAS-RalGDS_signaling_pathway','EGF-EGFR-RAS-RASSF1_signaling_pathway','EGF-ERBB2-RAS-ERK_signaling_pathway','EGF-Jak-STAT_signaling_pathway','FGF-FGFR-PI3K_signaling_pathway','FGF-FGFR-RAS-ERK_signaling_pathway','FLT3LG-FLT3-RAS-ERK_signaling_pathway','FLT3LG-FLT3-RAS-PI3K_signaling_pathway','FLT3LG-FLT3-STAT5_signaling_pathway','Hedgehog_signaling_pathway','HGF-MET-PI3K_signaling_pathway','HGF-MET-RAS-ERK_signaling_pathway','HIF-1_signaling_pathway','IGF2-IGF1R-PI3K_signaling_pathway','IGF-IGF1R-RAS-ERK_signaling_pathway','Intrinsic_apoptotic_pathway','KEAP1-NRF2_signaling_pathway','KITLG-KIT-PI3K_signaling_pathway','KITLG-KIT-RAS-ERK_signaling_pathway','MDM2-p21-Cell_cycle_G1-S','NOTCH_signaling_pathway','p15-Cell_cycle_G1-S','p16-Cell_cycle_G1-S','p27-Cell_cycle_G1-S','PDGF-PDGFR-PI3K_signaling_pathway','PDGF-PDGFR-PLCG-CAMK_signaling_pathway','PDGF-PDGFR-RAS-ERK_signaling_pathway','PTEN-PIP3-AKT_signaling_pathway','Telomerase_activity','TGFA-EGFR-PI3K_signaling_pathway','TGFA-EGFR-PLCG-PKC_signaling_pathway','TGFA-EGFR-RAS-ERK_signaling_pathway','TGF-beta_signaling_pathway','WNT_signaling_pathway','z1','z2','z3','z4','z5','z6','z7','z8','z9']



df_hm_p.drop('z1',axis=1,inplace=True)

df_hm_p.drop('z2',axis=1,inplace=True)


df_hm_p.drop('z3',axis=1,inplace=True)

df_hm_p.drop('z4',axis=1,inplace=True)


df_hm_p.drop('z5',axis=1,inplace=True)


df_hm_p.drop('z6',axis=1,inplace=True)


df_hm_p.drop('z7',axis=1,inplace=True)

df_hm_p.drop('z8',axis=1,inplace=True)


df_hm_p.drop('z9',axis=1,inplace=True)
df_hm_p

In [ ]:
df_hm_n=pd.DataFrame(heatmap_negative.reshape(heatmap_negative.shape[0],49))


df_hm_n.columns=['Androgen_receptor_signaling_pathway','Cytokine-Jak-STAT_signaling_pathway','DCC-apoptotic_pathway','EGF-EGFR-PI3K_signaling_pathway','EGF-EGFR-PLCG-CAMK_signaling_pathway','EGF-EGFR-RAS-ERK_signaling_pathway','EGF-EGFR-RAS-RalGDS_signaling_pathway','EGF-EGFR-RAS-RASSF1_signaling_pathway','EGF-ERBB2-RAS-ERK_signaling_pathway','EGF-Jak-STAT_signaling_pathway','FGF-FGFR-PI3K_signaling_pathway','FGF-FGFR-RAS-ERK_signaling_pathway','FLT3LG-FLT3-RAS-ERK_signaling_pathway','FLT3LG-FLT3-RAS-PI3K_signaling_pathway','FLT3LG-FLT3-STAT5_signaling_pathway','Hedgehog_signaling_pathway','HGF-MET-PI3K_signaling_pathway','HGF-MET-RAS-ERK_signaling_pathway','HIF-1_signaling_pathway','IGF2-IGF1R-PI3K_signaling_pathway','IGF-IGF1R-RAS-ERK_signaling_pathway','Intrinsic_apoptotic_pathway','KEAP1-NRF2_signaling_pathway','KITLG-KIT-PI3K_signaling_pathway','KITLG-KIT-RAS-ERK_signaling_pathway','MDM2-p21-Cell_cycle_G1-S','NOTCH_signaling_pathway','p15-Cell_cycle_G1-S','p16-Cell_cycle_G1-S','p27-Cell_cycle_G1-S','PDGF-PDGFR-PI3K_signaling_pathway','PDGF-PDGFR-PLCG-CAMK_signaling_pathway','PDGF-PDGFR-RAS-ERK_signaling_pathway','PTEN-PIP3-AKT_signaling_pathway','Telomerase_activity','TGFA-EGFR-PI3K_signaling_pathway','TGFA-EGFR-PLCG-PKC_signaling_pathway','TGFA-EGFR-RAS-ERK_signaling_pathway','TGF-beta_signaling_pathway','WNT_signaling_pathway','z1','z2','z3','z4','z5','z6','z7','z8','z9']

df_hm_n.drop('z1',axis=1,inplace=True)


df_hm_n.drop('z2',axis=1,inplace=True)


df_hm_n.drop('z3',axis=1,inplace=True)


df_hm_n.drop('z4',axis=1,inplace=True)


df_hm_n.drop('z5',axis=1,inplace=True)

df_hm_n.drop('z6',axis=1,inplace=True)


df_hm_n.drop('z7',axis=1,inplace=True)


df_hm_n.drop('z8',axis=1,inplace=True)


df_hm_n.drop('z9',axis=1,inplace=True)
df_hm_n

In [ ]:
from scipy.stats import ranksums

pvalue_list=[]
for col_p in df_hm_p.columns:
    pvalue_list.append(ranksums(df_hm_p[col_p],df_hm_n[col_p]).pvalue)
    print('Testing {}'.format(col_p))
    print(ranksums(df_hm_p[col_p],df_hm_n[col_p]))

In [ ]:
from statsmodels.stats.multitest import multipletests

In [ ]:
multipletests(pvals=pvalue_list,alpha=0.0001,method='bonferroni')

## Kaplan Meier Estimation

In [ ]:
pathways=pd.read_csv('D:/MyData/Pathway_Attention/data/All_Sub-pathways/allPathways.csv',sep='\,',engine='python',header=None,index_col=0)
pathways=pathways.fillna(0)
pathways=pathways.drop(pathways.index[40])
pathways

In [ ]:
exp=pd.read_hdf('data/LUAD/tcga_luad.h5',key='gene_expression')

In [ ]:
exp.index.name=''
exp

In [ ]:
exp.drop('geneName',axis=1,inplace=True)

In [ ]:
exp=exp.T
exp

In [ ]:
tcga_clinical=pd.read_hdf('data/LUAD/tcga_luad.h5',key='clinical')

In [ ]:
exp.to_csv('tmp/exp_tcga_luad.csv')

In [ ]:
pathway_exp=pd.read_csv('tmp/res_tcga_luad.csv',sep=',')
pathway_exp

In [ ]:
new_index=pathway_exp['Unnamed: 0']
new_index

In [ ]:
pathway_exp=pathway_exp.iloc[:,1:578]
pathway_exp

In [ ]:
pathway_exp=pathway_exp.set_index(new_index)
pathway_exp

In [ ]:
pathway_exp_new=pathway_exp.T.loc[[s for s in tcga_clinical[tcga_clinical['Pat_Died'].notna()].index]].T
pathway_exp_new

In [ ]:
pathway_exp_median=pathway_exp_new.median(axis=1)
pathway_exp_median.loc['Androgen_receptor_signaling_pathway']

In [ ]:
pathway_exp_mean=pathway_exp_new.mean(axis=1)
pathway_exp_mean.loc['Androgen_receptor_signaling_pathway']

In [ ]:
'''
Testing Androgen_receptor_signaling_pathway
RanksumsResult(statistic=-0.23275921813055575, pvalue=0.8159483823927877)
Testing Cytokine-Jak-STAT_signaling_pathway
RanksumsResult(statistic=0.4539680500044803, pvalue=0.6498518230163224)
Testing DCC-apoptotic_pathway
RanksumsResult(statistic=0.7844321748304738, pvalue=0.43278656528331594)
Testing EGF-EGFR-PI3K_signaling_pathway
RanksumsResult(statistic=5.15862025539502, pvalue=2.487763015351339e-07)
Testing EGF-EGFR-PLCG-CAMK_signaling_pathway
RanksumsResult(statistic=-5.223472833967088, pvalue=1.755982368170443e-07)
Testing EGF-EGFR-RAS-ERK_signaling_pathway
RanksumsResult(statistic=4.450685720690432, pvalue=8.559653154069017e-06)
Testing EGF-EGFR-RAS-RalGDS_signaling_pathway
RanksumsResult(statistic=-1.428792247474998, pvalue=0.15306395446310792)
Testing EGF-EGFR-RAS-RASSF1_signaling_pathway
RanksumsResult(statistic=2.764850041356797, pvalue=0.005694896932146102)
Testing EGF-ERBB2-RAS-ERK_signaling_pathway
RanksumsResult(statistic=6.006863580528522, pvalue=1.891465331811031e-09)
Testing EGF-Jak-STAT_signaling_pathway
RanksumsResult(statistic=4.64098306803329, pvalue=3.4675554099515115e-06)
Testing FGF-FGFR-PI3K_signaling_pathway
RanksumsResult(statistic=10.770782521957186, pvalue=4.729640954978687e-27)
Testing FGF-FGFR-RAS-ERK_signaling_pathway
RanksumsResult(statistic=4.686001171844996, pvalue=2.7859466793825962e-06)
Testing FLT3LG-FLT3-RAS-ERK_signaling_pathway
RanksumsResult(statistic=0.6393896195422852, pvalue=0.5225695001864655)
Testing FLT3LG-FLT3-RAS-PI3K_signaling_pathway
RanksumsResult(statistic=1.6758474354805644, pvalue=0.09376807998388782)
Testing FLT3LG-FLT3-STAT5_signaling_pathway
RanksumsResult(statistic=1.5225681293372373, pvalue=0.12786678839455717)
Testing Hedgehog_signaling_pathway
RanksumsResult(statistic=5.822341426313979, pvalue=5.802884929950376e-09)
Testing HGF-MET-PI3K_signaling_pathway
RanksumsResult(statistic=4.582710422615614, pvalue=4.589874536363513e-06)
Testing HGF-MET-RAS-ERK_signaling_pathway
RanksumsResult(statistic=3.511080733772921, pvalue=0.0004462888378208215)
Testing HIF-1_signaling_pathway
RanksumsResult(statistic=9.739721197958481, pvalue=2.0411358539152124e-22)
Testing IGF2-IGF1R-PI3K_signaling_pathway
RanksumsResult(statistic=8.091424273958896, pvalue=5.897105143616761e-16)
Testing IGF-IGF1R-RAS-ERK_signaling_pathway
RanksumsResult(statistic=3.8156985024672725, pvalue=0.00013579823579522977)
Testing Intrinsic_apoptotic_pathway
RanksumsResult(statistic=10.593550365625926, pvalue=3.1924879054542334e-26)
Testing KEAP1-NRF2_signaling_pathway
RanksumsResult(statistic=8.306621224461542, pvalue=9.846562875674444e-17)
Testing KITLG-KIT-PI3K_signaling_pathway
RanksumsResult(statistic=8.106240957968428, pvalue=5.220985624935596e-16)
Testing KITLG-KIT-RAS-ERK_signaling_pathway
RanksumsResult(statistic=6.275220710400826, pvalue=3.4913866761651527e-10)
Testing MDM2-p21-Cell_cycle_G1-S
RanksumsResult(statistic=10.458827417730959, pvalue=1.3349747474837594e-25)
Testing NOTCH_signaling_pathway
RanksumsResult(statistic=4.159843207736571, pvalue=3.184661404709197e-05)
Testing p15-Cell_cycle_G1-S
RanksumsResult(statistic=2.809442106331168, pvalue=0.004962744398402544)
Testing p16-Cell_cycle_G1-S
RanksumsResult(statistic=5.088560535477749, pvalue=3.6079161537819775e-07)
Testing p27-Cell_cycle_G1-S
RanksumsResult(statistic=5.916069970527626, pvalue=3.297249868352902e-09)
Testing PDGF-PDGFR-PI3K_signaling_pathway
RanksumsResult(statistic=6.196829564331231, pvalue=5.761182662836725e-10)
Testing PDGF-PDGFR-PLCG-CAMK_signaling_pathway
RanksumsResult(statistic=6.8924089727531115, pvalue=5.48553939617304e-12)
Testing PDGF-PDGFR-RAS-ERK_signaling_pathway
RanksumsResult(statistic=5.336325788212207, pvalue=9.484891589946548e-08)
Testing PTEN-PIP3-AKT_signaling_pathway
RanksumsResult(statistic=4.565574193825038, pvalue=4.981289736960464e-06)
Testing Telomerase_activity
RanksumsResult(statistic=2.2180055248133397, pvalue=0.02655445571841898)
Testing TGFA-EGFR-PI3K_signaling_pathway
RanksumsResult(statistic=-1.5901662915276855, pvalue=0.11179732648468485)
Testing TGFA-EGFR-PLCG-PKC_signaling_pathway
RanksumsResult(statistic=4.542189395420219, pvalue=5.567298859490183e-06)
Testing TGFA-EGFR-RAS-ERK_signaling_pathway
RanksumsResult(statistic=2.082619849838072, pvalue=0.03728588841669718)
Testing TGF-beta_signaling_pathway
RanksumsResult(statistic=5.4483740024312475, pvalue=5.083238167965829e-08)
Testing WNT_signaling_pathway
RanksumsResult(statistic=4.2312757194630315, pvalue=2.323696207065102e-05)
'''

In [ ]:
pathway='WNT_signaling_pathway'

In [ ]:
over_pat_list=[]
lower_pat_list=[]
for pat in pathway_exp.columns:
    if pat!='geneName':
        if pathway_exp[pat].loc[pathway]>=pathway_exp_median.loc[pathway]:
            over_pat_list.append(pat)
        if pathway_exp[pat].loc[pathway]<pathway_exp_median.loc[pathway]:
            lower_pat_list.append(pat)
print(len(over_pat_list))
print(len(lower_pat_list))

In [ ]:
tcga_clinical_over=tcga_clinical[tcga_clinical.index.isin(over_pat_list)]
tcga_clinical_lower=tcga_clinical[tcga_clinical.index.isin(lower_pat_list)]

In [ ]:
tcga_clinical_over=tcga_clinical_over[tcga_clinical_over['Pat_Overall_Survival_Months'].notna()]

In [ ]:
tcga_clinical_lower=tcga_clinical_lower[tcga_clinical_lower['Pat_Overall_Survival_Months'].notna()]

In [ ]:
len(tcga_clinical_over)

In [ ]:
len(tcga_clinical_lower)

In [ ]:
from lifelines import KaplanMeierFitter
timeline=[]
for i in range(0,120):
    timeline.append(i)

In [ ]:
from lifelines.statistics import logrank_test
from lifelines.utils import restricted_mean_survival_time
from lifelines.plotting import rmst_plot

T_p=tcga_clinical_over['Pat_Overall_Survival_Months']
E_p=tcga_clinical_over['Pat_Died']
kmf_positive = KaplanMeierFitter()
kmf_positive.fit(durations=T_p, event_observed=E_p,timeline=timeline,label='high expression')
ax=kmf_positive.plot()

T_n=tcga_clinical_lower['Pat_Overall_Survival_Months']
E_n=tcga_clinical_lower['Pat_Died']
kmf_negative=KaplanMeierFitter()
kmf_negative.fit(durations=T_n, event_observed=E_n,timeline=timeline,label='low expression')
ax=kmf_negative.plot(ax=ax)

results = logrank_test(T_p, T_n, event_observed_A=E_p, event_observed_B=E_n)
'''
rmst_positive= restricted_mean_survival_time(kmf_positive, t=900)
rmst_negative= restricted_mean_survival_time(kmf_negative, t=900)

ax = plt.subplot(311)
rmst_plot(kmf_negative, t=900, ax=ax)


ax = plt.subplot(312)
rmst_plot(kmf_positive, t=900, ax=ax)


ax = plt.subplot(313)
rmst_plot(kmf_negative, model2=kmf_positive, t=900, ax=ax)
'''
plt.legend(loc='lower left', bbox_to_anchor=(0.01,0.01),ncol=1,fancybox=False,shadow=False)
plt.title('p-value: {}'.format(results.p_value))

In [ ]:
km_list=[
  0.1473278086417022,
    0.8380701744826845,
    0.8319528662648101,
    0.2677993101053783,
    0.24169628087995332,
    0.7179620749675725,
    0.703163680749147,
    0.0859219589066487,
    0.39241596202194984,
    0.5052318610562971,
    0.7791758528948454,
    0.3145820343594065,
    0.5608911593256773,
    0.13371215603568046,
    0.055607757150250244,
    0.08211688594518443,
    0.08209434578894251,
    0.6043766622603884,
    0.06763922858669998,
    0.09180037673849839,
    0.5645998046988128,
    0.08829066942559403,
    0.5797097523758065,
    0.6068390134591286,
    0.43971302472109297,
    0.6069237664612511,
    0.34305073241509954,
    0.920532193088031,
    0.9688898828243467,
    0.19129624022109548,
    0.1734978667586194,
    0.9164063904670604,
    0.6603181590477675,
    0.4364572547979667,
    0.3046715590326145,
    0.010264399242450574,
    0.44457539496593124,
    0.7021987238671419,
    0.7287282043787895
]

In [ ]:
len(km_list)

In [ ]:
bonferroni_list=[2.33122806e-004, 4.02401335e-027, 3.23739631e-023, 4.87646841e-004,
        2.76578020e-198, 1.00000000e+000, 1.31111601e-065, 1.84895135e-101,
        1.00000000e+000, 1.14149466e-019, 2.99715772e-126, 3.42378213e-016,
        3.53590379e-090, 1.00000000e+000, 6.60107037e-030, 3.78770309e-045,
        9.39796431e-006, 8.51691632e-009, 1.90794832e-004, 2.64275675e-025,
        1.46343762e-110, 7.10822430e-026, 1.13006408e-050, 5.99467006e-030,
        6.06612561e-031, 1.39358934e-047, 9.37038855e-042, 4.67306885e-018,
        9.01713109e-036, 5.13895195e-001, 1.00000000e+000, 5.15421216e-027,
        2.64140205e-025, 8.83100030e-039, 1.00000000e+000,
        6.93672346e-001, 1.01377590e-074, 3.97996130e-013, 2.36223205e-003]

In [ ]:
from lifelines.utils import concordance_index

In [ ]:
print(concordance_index(km_list,bonferroni_list))